In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()


(6759, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,1.8857,0.0000,6.511986,1.881337,1.978520,0.048591,0.048591,1.8857,0.067659
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,6.0,1.9556,0.0886,6.992522,1.765439,1.961637,0.098099,0.098099,1.9556,0.088600
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.7824,0.0000,6.569075,1.798748,2.018901,0.110076,0.110076,1.7824,0.110076
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,6.0,1.8027,0.0586,6.290935,1.791712,1.973971,0.091129,0.091129,1.8027,0.067659
4,4,2,Göteborg Majorna OK 1,Göteborg Majorna OK,SWE,1,6.0,amanda hansson,1,Amanda Hansson,2.0,2.0053,0.0358,7.531656,1.880709,2.035837,0.077564,0.077564,2.0053,0.077564


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std", "num_runs"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std,num_runs
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,1.8857,0.067659,1.0
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.9556,0.088600,6.0
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.7824,0.110076,1.0
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.8027,0.067659,6.0
4,2,Göteborg Majorna OK 1,1,6.0,Amanda Hansson,2.0053,0.077564,2.0


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
1              Stora Tuna OK 1        Stora Tuna OK 1        Stora Tuna OK 1   
2        Göteborg Majorna OK 1  Göteborg Majorna OK 1  Göteborg Majorna OK 1   
3                   MS Parma 1             MS Parma 1             MS Parma 1   
4          Järla Orientering 1    Järla Orientering 1    Järla Orientering 1   
5               Alfta ÖSA OK 1         Alfta ÖSA OK 1         Alfta ÖSA OK 1   

                               dist                               name  \
leg                          4    1    2    3    4                   1   
team_id                                                                  
1              Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
2        Göteborg Majorna OK 1  6.0  5.7  7.3  7.9      Amanda Hansson   
3                   MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4          Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5               Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   

                              ... log_mean           log_std            \
leg                        2  ...        3       4         1         2   
team_id                       ...                                        
1           Magdalena Olsson  ...   1.7824  1.8027  0.067659  0.088600   
2              Runa Fremstad  ...   1.9581  1.9854  0.077564  0.087900   
3             Anni Heikkonen  ...   1.8669  1.9157  0.067659  0.108600   
4        Anna Dahlgren-Rosén  ...   1.8086  1.9289  0.112800  0.182322   
5            Josefine Heikka  ...   1.9000  1.8720  0.110100  0.117383   

                            num_runs                 
leg             3         4        1    2    3    4  
team_id                                              
1        0.110076  0.067659      1.0  6.0  1.0  6.0  
2        0.067659  0.097276      2.0  5.0  5.0  3.0  
3        0.128204  0.067659      4.0  6.0  1.0  7.0  
4        0.124320  0.087344      4.0  0.0  1.0  2.0  
5        0.095300  0.125945      6.0  2.0  6.0  3.0  

[5 rows x 24 columns]

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1697, 24)

(1682, 24)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,34.691978,71.617793,111.439271,157.906625,45.189117,89.222620,137.144526,186.554937,39.551333,79.872909,...,35.048827,42.044603,45.189117,47.901834,54.051577,54.694844,39.551333,40.257489,43.441498,47.948494
2,38.261758,77.704366,127.389117,181.009468,51.976932,97.725887,152.259965,214.337579,44.560778,87.220727,...,45.362427,47.496493,51.976932,50.613968,59.194300,69.578672,44.560778,42.516899,51.662974,57.522845
3,34.726960,70.282145,112.674124,164.598914,45.370361,90.575780,144.234530,198.997534,39.742391,79.494876,...,36.912838,46.929870,45.370361,49.065375,60.852691,61.116764,39.742391,39.694244,47.150453,53.675323
4,34.749353,68.279987,109.434446,161.969719,53.768658,102.362003,150.311326,207.134395,43.177172,83.441607,...,34.811823,45.812044,53.768658,56.396165,56.766470,64.396623,43.177172,39.871895,44.471798,54.417145
5,32.217419,66.473491,112.324456,159.716504,49.410975,90.770476,143.249855,199.735907,39.882379,77.820450,...,40.425356,40.068356,49.410975,47.302536,58.753685,65.430276,39.882379,37.675487,48.758719,51.271279


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4,num_runs_1,num_runs_2,num_runs_3,num_runs_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,1.78,1.80,0.07,0.09,0.11,0.07,1.0,6.0,1.0,6.0
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,1.96,1.99,0.08,0.09,0.07,0.10,2.0,5.0,5.0,3.0
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,1.87,1.92,0.07,0.11,0.13,0.07,4.0,6.0,1.0,7.0
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,1.81,1.93,0.11,0.18,0.12,0.09,4.0,0.0,1.0,2.0
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,1.90,1.87,0.11,0.12,0.10,0.13,6.0,2.0,6.0,3.0


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,35.048827,42.044603,45.189117,47.901834,54.051577,54.694844,39.551333,40.257489,43.441498,47.948494
2,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,Göteborg Majorna OK 1,6.0,5.7,7.3,7.9,Amanda Hansson,Runa Fremstad,...,45.362427,47.496493,51.976932,50.613968,59.194300,69.578672,44.560778,42.516899,51.662974,57.522845
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,36.912838,46.929870,45.370361,49.065375,60.852691,61.116764,39.742391,39.694244,47.150453,53.675323
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,34.811823,45.812044,53.768658,56.396165,56.766470,64.396623,43.177172,39.871895,44.471798,54.417145
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.425356,40.068356,49.410975,47.302536,58.753685,65.430276,39.882379,37.675487,48.758719,51.271279


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   
        2    Göteborg Majorna OK 1   5.7       Runa Fremstad    2.0081   
        3    Göteborg Majorna OK 1   7.3    Kristine Berglia    1.9581   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.067659       1.0    34.691978   45.189117   39.551333   
        2    0.088600       6.0    71.617793   89.222620   79.872909   
        3    0.110076       1.0   111.439271  137.144526  123.447979   
        4    0.067659       6.0   157.906625  186.554937  171.590677   
2       1    0.077564       2.0    38.261758   51.976932   44.560778   
        2    0.087900       5.0    77.704366   97.725887   87.220727   
        3    0.067659       5.0   127.389117  152.259965  139.042406   

               fin_mean  fin_sum_log_mean  fin_sum_log_std  mass_start  \
team_id leg                                                              
1       1     39.644208          3.677645         0.067824         0.0   
        2     80.053299          4.381138         0.055759         0.0   
        3    123.711380          4.816543         0.053067         0.0   
        4    171.774124          5.145269         0.042697         0.0   
2       1     44.695803          3.796847         0.077894         0.0   
        2     87.327424          4.467908         0.059292         0.0   
        3    139.171075          4.934676         0.045345         0.0   

             dark_during_leg  personal_start95  personal_end95  \
team_id leg                                                      
1       1                0.0         34.691978       45.189117   
        2                0.0         33.909699       47.901834   
        3                0.0         35.048827       54.051577   
        4                0.0         42.044603       54.694844   
2       1                0.0         38.261758       51.976932   
        2                0.0         35.641949       50.613968   
        3                0.0         45.362427       59.194300   

             personal_median  
team_id leg                   
1       1          39.551333  
        2          40.257489  
        3          43.441498  
        4          47.948494  
2       1          44.560778  
        2          42.516899  
        3          51.662974

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean  \
team_id leg                                                              
1       1          Stora Tuna OK 1   6.0       Tilda Östberg    1.8857   
        2          Stora Tuna OK 1   5.7    Magdalena Olsson    1.9556   
        3          Stora Tuna OK 1   7.3         Julia Jakob    1.7824   
        4          Stora Tuna OK 1   7.9  Tove Alexandersson    1.8027   
2       1    Göteborg Majorna OK 1   6.0      Amanda Hansson    2.0053   

              log_std  num_runs  fin_start95   fin_end95  fin_median  \
team_id leg                                                            
1       1    0.067659       1.0    34.691978   45.189117   39.551333   
        2    0.088600       6.0    71.617793   89.222620   79.872909   
        3    0.110076       1.0   111.439271  137.144526  123.447979   
        4    0.067659       6.0   157.906625  186.554937  171.590677   
2       1    0.077564       2.0    38.261758   51.976932   44.560778   

               fin_mean  ...  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg              ...                                                 
1       1     39.644208  ...         0.067824         0.0              0.0   
        2     80.053299  ...         0.055759         0.0              0.0   
        3    123.711380  ...         0.053067         0.0              0.0   
        4    171.774124  ...         0.042697         0.0              0.0   
2       1     44.695803  ...         0.077894         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           34.691978       45.189117        39.551333   
        2           33.909699       47.901834        40.257489   
        3           35.048827       54.051577        43.441498   
        4           42.044603       54.694844        47.948494   
2       1           38.261758       51.976932        44.560778   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:39:33.079965353 2019-06-15 14:34:41.518676519   
        2   2019-06-15 15:19:52.374566793 2019-06-15 15:11:37.067568541   
        3   2019-06-15 16:03:26.878745794 2019-06-15 15:51:26.356262207   
        4   2019-06-15 16:51:35.440620184 2019-06-15 16:37:54.397517681   
2       1   2019-06-15 14:44:33.646675348 2019-06-15 14:38:15.705471754   

                           fin_time_end95 personal_estimate_interval  
team_id leg                                                           
1       1   2019-06-15 14:45:11.347011089                  10.497139  
        2   2019-06-15 15:29:13.357202291                  13.992135  
        3   2019-06-15 16:17:08.671575785                  19.002751  
        4   2019-06-15 17:06:33.296226740                  12.650241  
2       1   2019-06-15 14:51:58.615939617                  13.715174  

[5 rows x 21 columns]

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Puskasilimät")]

for_print = for_print[[
 'leg',
 'name', 
 'num_runs', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,num_runs,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,,


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'num_runs',     
 'ind_log_mean', 
 'ind_log_std',
 'personal_start95', 
 'personal_median',
 'personal_end95',    
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

#for_web[for_web['team'].str.contains("Jyry")]
for_web[for_web['team'].str.contains("Downt")]

,team_id,leg,team,name,num_runs,ind_log_mean,ind_log_std,personal_start95,personal_median,personal_end95,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(6728, 17)

In [20]:
for_print[['num_runs', 'personal_median', "personal_estimate_interval"]].groupby('num_runs').agg(["mean"]).round(2)


,personal_median,personal_estimate_interval
,mean,mean
num_runs,,
